In [67]:
from scipy.stats import norm
import numpy as np
import yfinance as yf
import pandas as pd
from ta.volatility import AverageTrueRange
from datetime import datetime


def get_historical_data(stk):
    df = yf.download(stk,period='max', interval="1wk").dropna()
    df['year']=df.index.isocalendar().year.astype(int)
    return df


def trade_parameters(df, yield_perc):
    df["strike_price"] = df.Open.round(0)
    df["straddle_yield"] = yield_perc
    df["straddle_premium"] = df.strike_price * yield_perc
    
    df['ATR']=AverageTrueRange(high=df.High,low=df.Low,close=df.Close,window=5,fillna=False).average_true_range().rank(pct=True).shift(1)
    df['abs_change']=abs(df.Close-df.Open)
    df['abs_change_pct']=df.abs_change/df.Open
    df["profit_loss"] = df.straddle_premium - df.abs_change
    df["log_return"] = np.log((df.strike_price + df.profit_loss) / df.strike_price)
    df['pos_ret']=np.where(df.log_return>0,df.log_return,0)
    df['neg_ret']=np.where(df.log_return<0,df.log_return,0)
    df = df.iloc[1: , ::]
    
    lim=0.3
    df=df[df.ATR<lim]
    
    return df

def annual_group(df):
    df = df.groupby(["year"]).agg(
        {"log_return": "sum",'pos_ret':'sum','neg_ret':'sum'}
    )
    df['sharpe']=df.pos_ret/abs(df.neg_ret)
    return(df)
    
    
atm_option_data = pd.read_csv("historical_straddle_data/option_data_2022-09-02_2022-08-29.csv")

years=np.arange(2007, 2023, 1, dtype=int)
#print(years)

zeros=np.zeros((16,2),dtype=float)
#print(zeros)
blank=pd.DataFrame(columns=['log_return','sharpe'],index=years,data=zeros)
blank=blank.rename_axis('year')
#print(blank)
arr = []
limited=['HYG']
atm_option_data['yield_perc']=(atm_option_data.atm_call_last + atm_option_data.atm_put_last) / atm_option_data.strike
atm_option_data=atm_option_data[atm_option_data.symbol.isin(limited)]
for i,j in zip(atm_option_data.symbol,atm_option_data.yield_perc):
    
    stock = i
    yield_perc = j
    df = get_historical_data(stock)
    #print(df.tail)
    df = trade_parameters(df, yield_perc)
    #print(df.tail)
    df.to_csv(stock + ".csv")
    df = annual_group(df)

    df=df.iloc[::,[0,3]]
    #print(df)
    df=blank.add(df,fill_value=0)
    
    print(df)
    
    df.to_csv('out.csv')

[*********************100%***********************]  1 of 1 completed
      log_return       sharpe
year                         
2007    0.103893          inf
2008    0.000000     0.000000
2009    0.000000     0.000000
2010    0.000000     0.000000
2011    0.085319   104.196582
2012    0.000000     0.000000
2013    0.208533          inf
2014    0.366997    70.559768
2015    0.108197  3093.180302
2016    0.000000     0.000000
2017    0.511191   267.236580
2018    0.380620  1163.653224
2019    0.318279   234.282025
2020    0.094516   106.494299
2021    0.586371          inf
2022    0.003620          inf


/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind